In [1]:
import pandas as pd  
import numpy as np
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

#  Data Preparation

In [2]:
data = pd.read_csv("CAvideos.csv")

In [3]:
data = data[data.tags !='[none]']

In [4]:
data.columns.values

array(['video_id', 'trending_date', 'title', 'channel_title',
       'category_id', 'publish_time', 'tags', 'views', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'video_error_or_removed', 'description'],
      dtype=object)

In [5]:
data.shape

(38496, 16)

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.2, random_state=0)

train, val = train_test_split(train, test_size = 0.25, random_state=0)

In [7]:
train.shape

(23097, 16)

In [8]:
val.shape

(7699, 16)

In [9]:
test.shape

(7700, 16)

In [10]:
def tags_to_phrases( raw_tags ):
    # Function to convert a raw tags to a string of phrases
    # The input is a single string (a raw video tags), and 
    # the output is a single string (a preprocessed video tags)
     
    phrases = raw_tags.lower().split('|')                           
    
    # In Python, searching a set is much faster than searching
    # a list, so convert the stop words to a set
    # to improve execution time this conversion should be done once
    stops = set(stopwords.words("english"))                  
     
    # Remove stop words
    meaningful_words = [w for w in phrases if not w in stops]   
    
    # Join the words back into one string separated by space, 
    # and return the result.
    return( ",".join( meaningful_words ))  

In [11]:
# Initialize an empty list to hold the clean tags
clean_train_tags = []

In [12]:
for key in train['tags']:
    clean_train_tags.append( tags_to_phrases(key))

In [13]:
clean_train_tags[1]

'james corden,"the late late show","colbert","late night","late night show","stephen colbert","comedy","monologue","comedian","impressions","celebrities","carpool","karaoke","cbs","late late show","corden","joke","jokes","funny","funny video","funny videos","humor","celebrity","celeb","hollywood","famous"'

In [14]:
len(clean_train_tags)

23097

In [15]:
clean_val_tags = []

print ("Cleaning and parsing the validation set tags...\n")
for key in val['tags']:
    clean_val_tags.append(tags_to_phrases(key))

Cleaning and parsing the validation set tags...



In [16]:
clean_test_tags = []

print ("Cleaning and parsing the test set tags...\n")
for key in test['tags']:
    clean_test_tags.append(tags_to_phrases(key))

Cleaning and parsing the test set tags...



# Data Visualization

In [17]:
clean_data_tags = []

for key in data['tags']:
    clean_data_tags.append(tags_to_phrases(key))

In [18]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt


text = ','.join(clean_data_tags)

wordcloud = WordCloud(background_color='white',
                          width=800,
                          height=400
                         ).generate(text)

plt.figure(figsize=(24,14), dpi=1200)
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('cloud.png')

# Modeling

## 1. First attempt: using words in tags to predict 'views' 
Model: random forest + neuralnets


In [19]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
# Note that CountVectorizer comes with its own options to automatically do preprocessing, tokenization, and stop word removal -- for each of these, instead of specifying "None", we could have used a built-in method or specified our own function to use.

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_tags)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [20]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print (vocab)

['00', '00z', '01', '02', '03', '04', '05', '06', '07', '08282016ntflxuscan', '10', '100', '1000', '101', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '2016', '2017', '2018', '2019', '20th', '21', '22', '223', '23', '24', '25', '26', '27', '28', '29', '2k18', '2m', '2nd', '2stv', '30', '34', '360', '3d', '40', '43', '44', '4chan', '4k', '50', '500', '52', '5t', '60', '69', '6ix9ine', '720p', '73', '76ers', '90', '90s', '99', 'aangan', 'aaron', 'abandoned', 'abbas', 'abby', 'abc', 'abdul', 'about', 'abrams', 'abs', 'abuse', 'academy', 'access', 'accident', 'account', 'ace', 'achievement', 'acoustic', 'acting', 'action', 'activities', 'actor', 'actors', 'actress', 'actually', 'ad', 'adam', 'adams', 'adeeb', 'adidas', 'adidon', 'adkhawaja', 'administration', 'adobe', 'adrienne', 'ads', 'adult', 'adults', 'adventure', 'adventures', 'adventurous', 'advice', 'afc', 'affair', 'affairs', 'affordable', 'afghan', 'afghanistan', 'africa', 'african', 'afrique', 'after', 'aftermath',

In [21]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print (count, tag)

394 00
94 00z
132 01
105 02
113 03
167 04
169 05
37 06
34 07
46 08282016ntflxuscan
1450 10
427 100
49 1000
32 101
497 11
520 12
366 13
290 14
246 15
175 16
249 17
400 18
97 19
139 20
207 2016
2859 2017
6577 2018
62 2019
32 20th
218 21
119 22
30 223
150 23
256 24
111 25
91 26
106 27
70 28
76 29
30 2k18
38 2m
56 2nd
29 2stv
203 30
41 34
117 360
77 3d
54 40
44 43
31 44
48 4chan
68 4k
184 50
32 500
87 52
40 5t
45 60
62 69
144 6ix9ine
30 720p
38 73
40 76ers
41 90
44 90s
51 99
116 aangan
40 aaron
84 abandoned
69 abbas
34 abby
177 abc
31 abdul
220 about
29 abrams
441 abs
41 abuse
70 academy
48 access
66 accident
50 account
72 ace
36 achievement
32 acoustic
35 acting
720 action
63 activities
163 actor
47 actors
97 actress
78 actually
129 ad
395 adam
42 adams
52 adeeb
31 adidas
30 adidon
27 adkhawaja
46 administration
54 adobe
88 adrienne
108 ads
32 adult
128 adults
222 adventure
72 adventures
44 adventurous
110 advice
66 afc
31 affair
109 affairs
42 affordable
59 afghan
40 afghanistan
85 afric

40 coming
425 comment
72 commentary
71 comments
198 commercial
241 commercials
30 community
43 companies
115 company
28 comparison
171 competition
61 competitive
1028 compilation
320 complex
108 computer
37 con
43 conan
42 concealer
38 conceited
36 concert
163 conference
108 congress
132 conor
159 conservative
228 conspiracy
30 construction
54 contest
28 contestant
184 control
29 controversial
51 controversy
43 conway
309 cook
41 cookery
39 cookie
39 cookies
570 cooking
234 cool
122 cooper
35 cop
39 copper
62 cops
339 corden
48 correspondents
69 corruption
73 corvette
257 cosmetics
64 costell
34 costume
40 count
61 countdown
44 countries
129 country
36 county
161 couple
80 couples
111 court
155 cover
50 coverage
130 covers
144 cow
175 cowbelly
36 cowchop
121 cowell
111 coyote
41 crab
30 cracker
111 craft
180 crafts
109 crash
53 crawford
39 craziest
324 crazy
155 cream
47 crease
29 creator
37 creature
80 creatures
56 credits
39 creed
166 creepy
54 creepypasta
50 crew
56 crews
114 cricke

43 joseph
125 josh
57 joshua
41 journalism
135 joy
41 joyner
290 jr
185 jre
82 jtbc
39 juan
34 juanpa
90 judge
53 judges
50 judy
29 juhi
252 juice
35 julia
29 julian
35 julien
61 jumanji
75 jump
45 jumper
40 jumping
32 jumpscare
28 jun
58 june
34 jungkook
75 jungle
38 junior
45 junkies
138 jurassic
233 just
174 justice
383 justin
110 justkidding
94 justkiddingparty
121 ka
35 kaala
59 kambi
436 kanye
305 kapoor
63 kapuso
51 karachi
52 karan
52 karaoke
28 karate
311 kardashian
60 kardashians
38 kareeb
83 karim
37 karlous
84 karma
28 kasparian
31 kat
228 kate
34 katherine
42 kathie
94 katie
91 katy
33 kawhi
45 kbs
50 ke
30 keaton
60 keemstar
48 keeping
47 kehlani
46 keith
78 kek
53 keke
45 kellerman
297 kelly
39 kellyanne
45 ken
84 kendall
103 kendrick
50 kennedy
34 kenny
29 kerem
29 kermit
535 kevin
94 key
76 keys
66 khabib
65 khaled
69 khalid
432 khan
105 khloe
32 khoosat
69 ki
56 kia
557 kid
134 kidding
1380 kids
33 kil
209 kilauea
66 kill
49 killer
340 kim
295 kimmel
538 king
95 kingd

37 snack
38 snacks
48 snake
36 snap
90 snapchat
28 snapping
113 sneak
60 sneaker
34 sneakerhead
49 sneakers
29 sneha
31 sniper
452 snl
89 snow
34 snowmobile
33 snowmobiles
143 so
39 soap
146 soccer
196 social
109 society
64 soda
32 soft
58 software
31 sohneya
81 soho
30 soldier
114 solo
28 solved
32 something
124 son
40 sonam
2534 song
3913 songs
213 soni
31 sonic
41 sonu
347 sony
37 soot
40 soothouse
29 sopanam
31 sophie
48 sorrow
58 sorry
29 sosyete
62 soul
75 sound
81 sounds
48 soundtrack
95 soup
29 sous
161 south
1586 sp
329 space
29 spacex
36 spade
42 spaghetti
42 sparklez
55 speaker
166 special
265 speech
268 speed
39 spice
180 spicy
232 spider
124 spiderman
48 spinner
114 spinners
80 spirit
29 spoiled
33 spoiler
62 spoilers
49 spongebob
41 spoof
45 spooky
155 sport
821 sports
28 sportsnet
50 spot
29 spotify
69 spotlight
112 spring
33 sprint
80 spurs
90 spy
70 squad
36 square
43 squat
153 squeezie
83 squirrel
58 sreekumar
127 sri
353 sridevi
320 st
152 staff
29 stage
50 stan
393 

51 افغان
35 اكشن
42 الأب
146 الأحد
47 الاب
59 الاولى
220 البيت
48 التاسعة
77 الثامن
61 الثاني
29 الجديد
53 الحاضرة
34 الحكمي
390 الحلقة
64 الحياة
56 الدوبلكس
51 الدولي
34 الرابع
71 الروحي
47 الصعيد
134 الطوفان
99 العروسة
212 الكبير
69 المصري
107 الموسم
101 اليوم
59 امام
98 ايوب
54 بدر
44 برامج
31 برنامج
59 بــــاس
44 بي
53 تاج
120 جار
77 جد
122 جلال
73 حد
28 حسن
65 حلقات
40 حلقة
143 خفية
40 دراما
74 ذا
36 رامز
30 رانيا
224 رحيم
342 رمضان
47 زيزو
120 سابع
47 ستاره
62 سریال
43 سوسن
89 سي
38 شعبان
53 شهرزاد
31 شهلا
102 صلاح
47 عائلة
99 عادل
46 عامر
72 عبد
31 عز
38 عشر
33 على
43 عمر
74 عمرو
80 عندي
143 عوالم
154 فاهيتا
34 فدان
61 فصل
34 فلوس
79 فويس
101 في
28 قسمت
79 قناة
66 كل
47 كلبش
44 كواليس
35 كيدز
59 لا
31 ليفربول
150 ما
217 محمد
724 مسلسل
240 مسلسلات
102 مشاهدة
72 مصارعه
65 مصر
42 مصطفى
77 مفيده
77 مكتوب
47 نسر
77 نقلك
146 نهار
30 هنا
81 ياسر
77 ية
73 يهم
54 يوتيوب
32 يوم
52 ٢٠١٨
138 डब
72 परस
72 पहलव
85 रक
72 वस
85 हत
76 னவள
76 யம
33 வமகள
34 አዲስ
29 ክፍል
73 ዘመን
30 የኢትዮጵያ
46 ድራማ
78 ải

Random forest with n_estimators = 18

In [22]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestRegressor

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestRegressor(n_estimators = 18) 

# Fit the forest to the training set, using the bag of words as 
# features and the views labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["views"] )

Training the random forest...


In [23]:
# Get a bag of words for the test set, and convert to a numpy array
val_data_features = vectorizer.transform(clean_val_tags)
np.asarray(val_data_features)

# Use the random forest to make sentiment label predictions
print ("Predicting validation labels...\n")
result = forest.predict(val_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "views" column
output = pd.DataFrame( data={"video_id":val["video_id"], "views":val["views"], "views_preds":result} )


Predicting validation labels...



In [24]:
mse = np.mean((output['views'] - output['views_preds'])**2)

In [25]:
print (mse)

3621220309592.0537


Neural net

In [26]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,),max_iter=500)
neuralnets = mlp.fit(train_data_features, train["views"] )

/Users/CTM_DETH/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [27]:
result_n = neuralnets.predict(val_data_features)
output_n = pd.DataFrame( data={"video_id":val["video_id"], "views":val["views"], "views_preds":result_n} )

In [28]:
output_n

,video_id,views,views_preds
28837,O5ROSS9ReUY,1269933,557678
26082,yWIJMEt5EvE,162973,174027
37336,AUPeHXijgH8,2772379,3041434
10742,PdrZmtKm52I,248124,224931
14574,m3Jt54qo_do,307361,195517
1380,ojFLLEdXIXk,46672,355039
2023,LgmxMuW6Fsc,411462,2618344
27315,j8giXiRyGbg,279863,465835
19648,7KPcfdbVTic,1150062,668744
301,eAyxMv4IBH8,399491,273860


In [29]:
mse_n = np.mean((output_n['views'] - output_n['views_preds'])**2)

In [30]:
mse_n

7185963903077.183

Ensemble method: averaging

In [31]:
ensem = (output_n['views_preds']  + output['views_preds']) /2

In [32]:
mse_two = np.mean((output_n['views'] - ensem)**2)

In [33]:
mse_two

4560243731076.916

## Second attempt: using phrases in tags to predict 'views'
Model: random forest + neuralnets


In [34]:
k = ','.join(clean_train_tags)
k = k.split(",")

In [35]:
from collections import Counter
import re
word_count = Counter(k)

In [36]:
top5000 = word_count.most_common(5000)
top5000

[('"funny"', 2336),
 ('"comedy"', 1764),
 ('"news"', 1215),
 ('"politics"', 859),
 ('"donald trump"', 727),
 ('"music"', 693),
 ('"2018"', 691),
 ('"video"', 677),
 ('"entertainment"', 660),
 ('"food"', 652),
 ('"trump"', 631),
 ('"humor"', 624),
 ('"review"', 622),
 ('"talk show"', 563),
 ('"reaction"', 528),
 ('"interview"', 528),
 ('"television"', 512),
 ('"rap"', 492),
 ('"drama"', 491),
 ('"comedian"', 463),
 ('"family friendly"', 462),
 ('"tv"', 455),
 ('"2017"', 452),
 ('"trailer"', 441),
 ('"how to"', 431),
 ('"sports"', 411),
 ('"funny videos"', 410),
 ('"react"', 409),
 ('"hollywood"', 403),
 ('"movie"', 401),
 ('"basketball"', 396),
 ('"celebrity"', 392),
 ('"jokes"', 381),
 ('"family"', 380),
 ('"new"', 374),
 ('"animation"', 369),
 ('"best"', 367),
 ('"show"', 362),
 ('"science"', 361),
 ('"vlog"', 358),
 ('"cooking"', 357),
 ('"celebrities"', 346),
 ('"highlights"', 335),
 ('"hip hop"', 335),
 ('"diy"', 334),
 ('"fun"', 333),
 ('"podcast"', 332),
 ('"canada"', 330),
 ('"g

In [37]:
lst = []
for value, count in top5000:
    lst.append(value)

In [38]:
freq_train = []
for word in lst:
    l = []
    for i in clean_train_tags:
        splits = i.split(",")
        if word in splits:
            l.append(1)
        else:
            l.append(0)
    l = np.asarray(l)
    freq_train.append(l)

In [39]:
freq__train_table = pd.DataFrame(freq_train)
freq__train_table = pd.DataFrame.transpose(freq__train_table)
freq__train_array = freq__train_table.values

Random Forest

In [40]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestRegressor

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestRegressor(n_estimators = 5) 

# Fit the forest to the training set, using the bag of words as 
# features and the views labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(freq__train_array, train["views"] )

Training the random forest...


In [41]:
freq = []
for word in lst:
    l = []
    for i in clean_val_tags:
        splits = i.split(",")
        if word in splits:
            l.append(1)
        else:
            l.append(0)
    l = np.asarray(l)
    freq.append(l)
    
freq_array = np.asarray(freq)
freq_table = pd.DataFrame(freq)
freq_table = pd.DataFrame.transpose(freq_table)
freq_array = freq_table.values

In [42]:
result = forest.predict(freq_array)

# Copy the results to a pandas dataframe with an "id" column and
# a "views" column
output = pd.DataFrame( data={"video_id":val["video_id"], "views":val["views"], "views_preds":result} )

In [43]:
mse = np.mean((output['views'] - output['views_preds'])**2)

In [44]:
print (mse)

4095120998712.8267


Neural network

In [45]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,),max_iter=500)
neuralnets = mlp.fit(freq__train_array, train["views"] )

/Users/CTM_DETH/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [46]:
result_n = neuralnets.predict(freq_array)

output_n = pd.DataFrame( data={"video_id":val["video_id"], "views":val["views"], "views_preds":result} )

In [47]:
mse_n = np.mean((output_n['views'] - output_n['views_preds'])**2)

In [48]:
mse_n

4095120998712.8267

Ensemble : averaging

In [49]:
ensem = (output_n['views_preds']  + output['views_preds']) /2

In [50]:
mse_two = np.mean((output_n['views'] - ensem)**2)

In [51]:
mse_two

4095120998712.8267

Since the model using words in our first attempt has better performance on the validation data, we choose this model to make our prediction.

# Use the test data to evaluate the performance of the chosen model

In [52]:
test_data_features = vectorizer.transform(clean_test_tags)
np.asarray(test_data_features)


print ("Predicting test labels...\n")
result = forest.predict(test_data_features)
output = pd.DataFrame( data={"video_id":test["video_id"], "views":test["views"], "views_preds":result} )
result_n = neuralnets.predict(test_data_features)
output_n = pd.DataFrame( data={"video_id":test["video_id"], "views":test["views"], "views_preds":result_n} )

ensem = (output_n['views_preds']  + output['views_preds']) /2

mse_two = np.mean((output_n['views'] - ensem)**2)
mse_two

Predicting test labels...



22576053619442.543